# Banka z SQLAlchemy

Knjižnico `sqlalchemy` namestimo z ukazom
```bash
python -m pip install sqlalchemy
```
oziroma (če nimamo administratorskih pravic)
```bash
python -m pip install --user sqlalchemy
```

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from sqlalchemy import create_engine

Definirajmo naše razrede (entitetne tipe).

In [2]:
Base = declarative_base()

class Kraj(Base):
    # Tako se bo imenavala tabela v bazi
    __tablename__ = 'kraj'
    
    # Atributi (lastnosti), ki sledijo se preslikajo v stolpce
    posta = Column(Integer, primary_key=True)
    kraj = Column(String(250))

class Oseba(Base):
    __tablename__ = 'oseba'
    
    id = Column(Integer, primary_key=True)               # Primarni ključ
    ime = Column(String(250), nullable=False)            # Ne sme biti NULL
    kraj_id = Column(Integer, ForeignKey('kraj.posta')) # tuji ključ
    # Lastnost, ki odraža povezavo do tabele kraj.
    # Ker je enolično določena, ne rabimo dodatnih parametrov.
    kraj = relationship(Kraj)

Generirajmo podatkovno bazo v SQLite (datoteko `banka-orm.db`) oziroma se z njo povežimo, če datoteka že obstaja.

Če želimo izpise stavkov SQL, nastavimo parameter `echo=True`. Tega v produkciji **ne** nastavimo.

In [3]:
engine = create_engine('sqlite:///banka-orm.db', echo=True)

Povežimo `engine` na razred `Base`.

In [4]:
Base.metadata.bind = engine

Tabele na bazi v tem trenutku še niso ustvarjene. Ustvarimo jih z naslednjim klicem.

In [5]:
Base.metadata.create_all(engine)

2024-05-13 17:22:47,667 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:22:47,668 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("kraj")
2024-05-13 17:22:47,668 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:22:47,669 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("kraj")
2024-05-13 17:22:47,670 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:22:47,671 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("oseba")
2024-05-13 17:22:47,671 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:22:47,672 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("oseba")
2024-05-13 17:22:47,673 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-13 17:22:47,674 INFO sqlalchemy.engine.Engine 
CREATE TABLE kraj (
	posta INTEGER NOT NULL, 
	kraj VARCHAR(250), 
	PRIMARY KEY (posta)
)


2024-05-13 17:22:47,675 INFO sqlalchemy.engine.Engine [no key 0.00063s] ()
2024-05-13 17:22:47,688 INFO sqlalchemy.engine.Engine 
CREATE TABLE oseba (
	id INTEGER NOT NULL,

Odprimo generator sej in ustvarimo novo sejo.

In [6]:
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

Ustvarimo nov objekt razreda `Kraj` in ga dodamo v sejo.

In [7]:
nov_kraj = Kraj(posta=1000, kraj='Ljubljana')
session.add(nov_kraj)

Metoda `commit` shrani (spremenjene) objekte v seji v podatkovno bazo.

In [8]:
session.commit()

2024-05-13 17:25:58,420 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:25:58,423 INFO sqlalchemy.engine.Engine INSERT INTO kraj (posta, kraj) VALUES (?, ?)
2024-05-13 17:25:58,423 INFO sqlalchemy.engine.Engine [generated in 0.00092s] (1000, 'Ljubljana')
2024-05-13 17:25:58,425 INFO sqlalchemy.engine.Engine COMMIT


Ustvarimo nov objekt razreda `Oseba` in ga preko lastnosti `kraj` povežemo s prej ustvarjenim krajem.

In [9]:
nova_oseba = Oseba(ime='Janez', kraj=nov_kraj)
session.add(nova_oseba)
session.commit()

2024-05-13 17:26:37,731 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:26:37,733 INFO sqlalchemy.engine.Engine SELECT kraj.posta AS kraj_posta, kraj.kraj AS kraj_kraj 
FROM kraj 
WHERE kraj.posta = ?
2024-05-13 17:26:37,735 INFO sqlalchemy.engine.Engine [generated in 0.00137s] (1000,)
2024-05-13 17:26:37,736 INFO sqlalchemy.engine.Engine INSERT INTO oseba (ime, kraj_id) VALUES (?, ?)
2024-05-13 17:26:37,737 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ('Janez', 1000)
2024-05-13 17:26:37,738 INFO sqlalchemy.engine.Engine COMMIT


Izvedimo poizvedbo `SELECT * FROM oseba`.

In [10]:
session.query(Oseba).all()

2024-05-13 17:27:47,327 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:27:47,330 INFO sqlalchemy.engine.Engine SELECT oseba.id AS oseba_id, oseba.ime AS oseba_ime, oseba.kraj_id AS oseba_kraj_id 
FROM oseba
2024-05-13 17:27:47,331 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()


Vrnimo prvi kraj in ga izpišimo.

In [11]:
kraj1 = session.query(Kraj).first()
print(kraj1)
print("KRAJ:", kraj1.kraj)

2024-05-13 17:28:08,367 INFO sqlalchemy.engine.Engine SELECT kraj.posta AS kraj_posta, kraj.kraj AS kraj_kraj 
FROM kraj
 LIMIT ? OFFSET ?
2024-05-13 17:28:08,368 INFO sqlalchemy.engine.Engine [generated in 0.00148s] (1, 0)
KRAJ: Ljubljana


Poiščimo vse osebe v kraju `kraj1` in jih izpišimo kot objekte.

In [12]:
session.query(Oseba).filter(Oseba.kraj == kraj1).all()

2024-05-13 17:28:51,085 INFO sqlalchemy.engine.Engine SELECT oseba.id AS oseba_id, oseba.ime AS oseba_ime, oseba.kraj_id AS oseba_kraj_id 
FROM oseba 
WHERE ? = oseba.kraj_id
2024-05-13 17:28:51,087 INFO sqlalchemy.engine.Engine [generated in 0.00173s] (1000,)


Poiščimo eno tako osebo ter jo vrnimo in izpišimo.

In [16]:
oseba1 = session.query(Oseba).filter(Oseba.kraj == kraj1).one()
print(oseba1)
print("IME:", oseba1.ime)
print("KRAJ:", oseba1.kraj)

2024-05-13 17:32:09,968 INFO sqlalchemy.engine.Engine SELECT oseba.id AS oseba_id, oseba.ime AS oseba_ime, oseba.kraj_id AS oseba_kraj_id 
FROM oseba 
WHERE ? = oseba.kraj_id
2024-05-13 17:32:09,969 INFO sqlalchemy.engine.Engine [cached since 198.9s ago] (1000,)
IME: Janez
KRAJ: <__main__.Kraj object at 0x7f9cfd047580>
